# LDA

In [1]:
import pandas as pd
import os
import csv

# Identify the working directory and data files
working_directory = './28-LDA'

# Create the working directory if needed
try:
    os.makedirs(working_directory, exist_ok=True)
except OSError as error:
    print(f"Error creating {working_directory}: {error}")

In [2]:
%pip install --quiet --upgrade nltk gensim spacy

Note: you may need to restart the kernel to use updated packages.


In [3]:
!python -m nltk.downloader --quiet 'all'
!python -m spacy download en_core_web_sm > /dev/null

/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))


In [4]:
# Read the preprocessed data into a dataframe
import pickle

with open('./21-data_preprocessing/dataframe.pickle', 'rb') as f:
    df = pickle.load(f)

In [5]:
df.shape

(5736, 20)

In [6]:
df.head(1)

,ROW_ID,FOI_TEXT,DEVICE_PROBLEM_CODE,DEVICE_PROBLEM_TEXT,GENERIC_NAME,DEVICE_REPORT_PRODUCT_CODE,UDI-DI,UDI-PUBLIC,DATE_OF_EVENT,REPORTER_OCCUPATION_CODE,REPORT_DATE,EVENT_LOCATION,SOURCE_TYPE,TOKENIZED_TEXT,NOPUNCT_TEXT,NOSTOPWORDS_TEXT,NODIGITS_TEXT,POS_TEXT,LEMMATIZED_TEXT,STEMMED_TEXT
0,1969025,IT WAS REPORTED THAT THE TRANSMITTER LOST CONN...,3283,Wireless Communication Problem,CONTINUOUS GLUCOSE MONITOR,QBJ,,,07/30/2020,000,,I,CONSUMER,"[it, was, reported, that, the, transmitter, lo...","[it, was, reported, that, the, transmitter, lo...","[reported, transmitter, lost, connection, pump...","[reported, transmitter, lost, connection, pump...","[(reported, VBN), (transmitter, NN), (lost, VB...","[report, transmitter, lose, connection, pump, ...","[report, transmitt, lost, connect, pump, great..."


In [26]:
import gensim
from gensim.utils import simple_preprocess

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data = df['FOI_TEXT'].tolist()
data_words = list(sent_to_words(data))

print(data_words[:1][0][:30])

['it', 'was', 'reported', 'that', 'the', 'transmitter', 'lost', 'connection', 'with', 'the', 'pump', 'for', 'greater', 'than', 'hour', 'the', 'transmitter', 'ultimately', 'regained', 'connection', 'with', 'the', 'pump', 'no', 'additional', 'patient', 'or', 'event', 'information', 'was']


In [27]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [28]:
# NLTK Stop words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [35]:
import spacy

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1][0][:30])

['report', 'transmitter', 'lose', 'connection', 'pump', 'great', 'hour', 'transmitter', 'ultimately', 'regain', 'connection', 'pump', 'additional', 'patient', 'event', 'information', 'available']


In [36]:
data_words_bigrams[0]

['reported',
 'transmitter',
 'lost',
 'connection',
 'pump',
 'greater',
 'hour',
 'transmitter',
 'ultimately',
 'regained',
 'connection',
 'pump',
 'additional',
 'patient',
 'event',
 'information',
 'available']

In [37]:
import gensim.corpora as corpora

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 2), (10, 1), (11, 1), (12, 2), (13, 1)]


In [38]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

In [39]:
from pprint import pprint

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.091*"perform" + 0.063*"pass" + 0.055*"transmitter" + 0.050*"report" + '
  '0.050*"fail" + 0.037*"evaluate" + 0.033*"test" + 0.033*"inspection" + '
  '0.033*"visual" + 0.032*"error"'),
 (1,
  '0.092*"transmitter" + 0.069*"pump" + 0.066*"connection" + 0.058*"patient" + '
  '0.052*"report" + 0.051*"information" + 0.050*"additional" + 0.048*"event" + '
  '0.048*"available" + 0.044*"lose"'),
 (2,
  '0.061*"device" + 0.054*"report" + 0.041*"loss" + 0.037*"operate" + '
  '0.031*"system" + 0.030*"mobile" + 0.029*"share" + 0.026*"support" + '
  '0.026*"determined" + 0.025*"signal"'),
 (3,
  '0.098*"loss" + 0.087*"signal" + 0.079*"report" + 0.056*"hour" + '
  '0.040*"probable" + 0.040*"cause" + 0.040*"allegation" + '
  '0.040*"intervention" + 0.040*"occur" + 0.039*"medical"'),
 (4,
  '0.089*"glucose" + 0.077*"bg" + 0.059*"reading" + 0.057*"read" + '
  '0.044*"blood" + 0.043*"meter" + 0.040*"monitor" + 0.039*"continuous" + '
  '0.039*"inaccuracy" + 0.039*"sensor"'),
 (5,
  '0.083*"repor

In [40]:
from gensim.models import CoherenceModel

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.6147046874725499


In [41]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [42]:
import numpy as np
import tqdm

grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 1
max_topics = 25
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]

corpus_title = ['75% Corpus', '100% Corpus']

model_results = {
    'Validation_Set': [],
    'Topics': [],
    'Alpha': [],
    'Beta': [],
    'Coherence': []
}

# Can take a long time to run
if True:
    pbar = tqdm.tqdm(total=(len(beta)*len(alpha)*len(topics_range)*len(corpus_title)))
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv(f"{working_directory}/lda_tuning_results.csv", index=False)
    pbar.close()

100%|██████████| 540/540 [49:47<00:00,  5.53s/it]


In [43]:
model_results_df = pd.DataFrame(model_results)

In [51]:
import pickle
import datetime

with open(f"{working_directory}/model_results_df-{datetime.datetime.now():%Y-%m-%d-%s}.pickle", 'wb') as f:
    pickle.dump(model_results_df, f)

/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentatio

In [44]:
model_results_df

,Validation_Set,Topics,Alpha,Beta,Coherence
0,75% Corpus,2,0.01,0.01,0.548065
1,75% Corpus,2,0.01,0.31,0.550543
2,75% Corpus,2,0.01,0.61,0.570513
3,75% Corpus,2,0.01,0.91,0.591976
4,75% Corpus,2,0.01,symmetric,0.570513
...,...,...,...,...,...
535,100% Corpus,10,asymmetric,0.01,0.639115
536,100% Corpus,10,asymmetric,0.31,0.617715
537,100% Corpus,10,asymmetric,0.61,0.656768
538,100% Corpus,10,asymmetric,0.91,0.656947


In [46]:
num_topics = 8
alpha      = 0.01
beta       = 0.9

lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=alpha,
                                           eta=beta)

In [48]:
%pip install --upgrade --quiet pyldavis

Note: you may need to restart the kernel to use updated packages.


In [49]:
import pyLDAvis.gensim_models as gensimvis
import pickle 
import pyLDAvis

# Visualize the topics
pyLDAvis.enable_notebook()

LDAvis_data_filepath = os.path.join(f"{working_directory}//ldavis_tuned_"+str(num_topics))

# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

pyLDAvis.save_html(LDAvis_prepared, f"{working_directory}/ldavis_tuned_"+ str(num_topics) +'.html')

LDAvis_prepared

/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.249329 -0.009077       1        1  28.259379
6     -0.220100  0.021248       2        1  21.419695
7     -0.179047 -0.120912       3        1  20.035885
0      0.098904 -0.153256       4        1   7.393600
4      0.227860 -0.147362       5        1   7.189953
5      0.171524  0.268950       6        1   6.220324
2     -0.039582  0.188691       7        1   5.427261
1      0.189770 -0.048281       8        1   4.053903, topic_info=            Term         Freq        Total Category  logprob  loglift
12   transmitter  3647.000000  3647.000000  Default  30.0000  30.0000
19    evaluation  2242.000000  2242.000000  Default  29.0000  29.0000
26       product  2313.000000  2313.000000  Default  28.0000  28.0000
47        device  1084.000000  1084.000000  Default  27.0000  27.0000
17         datum  2904.000000  2904.000000  Default  26.0000  26.0000
..           ...          ...          ...      ...      ...      ...
185      support    35.891406   258.202753   Topic8  -4.9498   1.2322
204   reportedly    32.662470   344.741684   Topic8  -5.0441   0.8489
143     multiple    25.910684   158.187098   Topic8  -5.2757   1.3964
24         occur    32.164085  3900.718900   Topic8  -5.0595  -1.5926
22          loss    27.555809  3033.940971   Topic8  -5.2141  -1.4959

[401 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
341       1  0.017943  accurate
341       2  0.017943  accurate
341       3  0.771545  accurate
341       4  0.017943  accurate
341       5  0.143543  accurate
...     ...       ...       ...
173       4  0.003933      zone
173       5  0.003933      zone
173       6  0.003933      zone
173       7  0.003933      zone
173       8  0.003933      zone

[2024 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 7, 8, 1, 5, 6, 3, 2])